In [1]:
# General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from random import randrange
import math
from queue import PriorityQueue

In [2]:
# dictionaries with the structure source -> array of followings
dictionary = defaultdict(list)
# simple datafram to store the sources and the count of source followings
data = pd.DataFrame([])

file = open('data/train.txt', 'r') 
lines = file.readlines() 
count = 0
for line in lines: 
    count = count + 1
    split_string = list(map(int,line.strip().split("\t")))
    dictionary[split_string[0]] = []
    if (len(split_string) >1):
        dictionary[split_string[0]] = sorted(split_string[1:len(split_string)])
        
    data = data.append(pd.DataFrame({'Source': split_string[0], 'Source_Followings': len(dictionary[split_string[0]]) }, index=[0]), ignore_index=True)


In [13]:
data.tail(10)

,Source,Source_Followings
19990,930244,1328
19991,4230790,135
19992,837929,0
19993,4592066,75
19994,4778890,664
19995,585576,64
19996,505961,539
19997,125824,45
19998,896087,219
19999,3547024,63


In [3]:
# function to calculate followers
def calculateFollowers(data,node):
    followers = 0
    for j in range(0,19999):
        if node in dictionary[data.at[j,'Source']]:
            followers = followers + 1
    
    return followers

# function to calculate followers
def calculateFollowers2(data,node):
    followers = 0
    for j in range(0,19999):
        common_elements = set(dictionary[data.at[j,'Source']]).intersection([node]) 
        followers = followers + len(common_elements)
    
    return followers

In [15]:
calculateFollowers2(data,4778890)

46

In [16]:
calculateFollowers(data,4778890)

46

In [17]:
train_data ={}
search_table = []
for line in open("data/train.txt"):
    r = line.split( )
    search_table.append(r)
    sink_ls = r[1:]
    train_data[r[0]] = sink_ls

In [18]:
def dijkstra(data,source,sink):
    visit ={}
    dis_dict = defaultdict(lambda: math.inf)
    dis_dict[source] =0
    expand_queue = PriorityQueue()
    expand_queue.put((0,source))
    while not expand_queue.empty():
        i,source = expand_queue.get()
        visit[source] = i
        if source == sink:
                return i
        if source in data:
            for s in data.get(source):
                if s not in visit or visit[s] > i+1:
                    expand_queue.put((i+1,s))

    return math.inf

In [20]:
# random selection of 2000 real edges
real_edges = pd.DataFrame([])
i = 0
while i < 2000:
    if i % 100 == 0:
        print(i)
    x = randrange(0, 19999)
    source = data.at[x,'Source']
    source_followings = data.at[x,'Source_Followings']
    if (source_followings > 0):
        i += 1
        y = random.randint(0, source_followings-1)
        sink = dictionary[source][y]
        source_followers = calculateFollowers(data,source)
        sink_followers = calculateFollowers(data,sink)
        real_edges = real_edges.append(pd.DataFrame({'Source': source,'Sink': sink,
                                                     'Source_Followings': source_followings,
                                                     'Source_Followers': source_followers,
                                                     'Sink_Followers': sink_followers},
                                                    index=[0]), ignore_index=True)

0


KeyboardInterrupt: 

In [ ]:
real_edges.head(10)

In [ ]:
# Saving real edges
real_edges.to_csv("model_data/real_edges.csv", sep='\t', index=False)

In [ ]:
# Reading real edges if they exist
real_edges = pd.read_csv("model_data/real_edges.csv", sep='\t')

In [4]:
# random creation of 2000 fake edges

#minimum = data['Source'].min()
#maximum = data['Source'].max()

fake_edges = pd.DataFrame([])
i = 0
while i < 4000:
    if i % 100 == 0:
        print(i)
    
    x = randrange(0, 19999)
    source = data.at[x,'Source']
    source_followings = data.at[x,'Source_Followings']
    
    y = randrange(0, 19999)
    source_gen = data.at[y,'Source']
    source_gen_followings = data.at[y,'Source_Followings']
    
    if (source_gen_followings > 0):
        z = random.randint(0, source_gen_followings-1)
        sink = dictionary[source_gen][z]

        # It is not real, is added
        if sink not in dictionary[source]:
            i += 1
            source_followers = calculateFollowers(data,source)
            sink_followers = calculateFollowers(data,sink)
            fake_edges = fake_edges.append(pd.DataFrame({'Source': source,'Sink': sink,
                                                         'Source_Followings': source_followings,
                                                         'Source_Followers': source_followers,
                                                         'Sink_Followers': sink_followers},
                                                        index=[0]), ignore_index=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [5]:
fake_edges.head()

,Source,Sink,Source_Followings,Source_Followers,Sink_Followers
0,2630160,3382945,10,4,115
1,37351,320447,1376,84,1
2,4762830,3123521,2352,1043,5
3,323641,981857,0,73,25
4,404163,4022635,33,12,30


In [7]:
fake_edges.to_csv("data_generatedgit /fake_edges3.csv", sep='\t', index=False)

In [ ]:
# Reading test-public data
test_public = pd.read_csv('data/test-public.txt', sep='\t')

# Just for exploration: number of sinks that are source in train file.
count = 0
for index, row in test_public.iterrows():
    if ( (data['Source'] == row['Sink']).any() ):
        count = count + 1

print(count)   

In [ ]:
# calculation of features for test data
test_data = pd.DataFrame([])
for index, row in test_public.iterrows():
    if index % 100 == 0:
        print(index)
    source = row['Source']
    source_followings = data.loc[data['Source'] == source, 'Source_Followings'].iloc[0]
    sink = row['Sink']

    source_followers = calculateFollowers(data,source)
    sink_followers = calculateFollowers(data,sink)




    test_data = test_data.append(pd.DataFrame({'Id': row['Id'], 'Source': source,'Sink': sink,
                                                 'Source_Followings': source_followings,
                                                 'Source_Followers': source_followers,
                                                 'Sink_Followers': sink_followers},
                                                index=[0]), ignore_index=True) 

In [ ]:
test_data.head(10)

In [ ]:
# This was part of the first exploration, maybe it is useful for the report
data.nlargest(50,'followers')
data.hist(column='Source_Followings',bins=100, range=(1,500))

# 